In [1]:
# Begin developing an image filter that automatically filters the image
# Inspired by the image sliders created using CV2

import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np
import cv2
from glob import glob
from sklearn.pipeline import Pipeline, make_pipeline

# sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Data folders
TRAIN_DATA_FOLDER = '/mnt/data/Plant-Seedlings-Classification/plant-seedlings-classification/train'
TEST_DATA_FOLDER = '/mnt/data/Plant-Seedlings-Classification/plant-seedlings-classification/test'

In [3]:
for file in os.listdir(TRAIN_DATA_FOLDER):
    print(file)

Common Chickweed
Scentless Mayweed
Loose Silky-bent
Common wheat
Sugar beet
Black-grass
Maize
Fat Hen
Shepherds Purse
Cleavers
Small-flowered Cranesbill
Charlock


In [4]:
# Default Filters

lower_HSV = np.array([35, 100, 5]) 
upper_HSV = np.array([200, 255, 255])

# Definitions

# HSV Definitions
def HSV_mask(image, lower_HSV, upper_HSV):
    mask = cv2.inRange(image, lower_HSV, upper_HSV)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    return mask
    
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp
 
def process_image(image, lower_HSV=lower_HSV, upper_HSV=upper_HSV):
    image = sharpen_image(image)
    mask = HSV_mask(image, lower_HSV, upper_HSV)
    output = cv2.bitwise_and(image, image, mask = mask)
    (contours, heiarchy) = cv2.findContours(mask, cv2.RETR_TREE, 
    cv2.CHAIN_APPROX_SIMPLE)
    return output, contours, heiarchy

# Sliders for adjusting HSV on sample image 
def LowerH(val, lower_HSV=lower_HSV):
    global sample_image
    lower_HSV[0] = val
    image, contours, heiarchy = process_image(sample_image, lower_HSV)
    cv2.drawContours(image, contours, -1, (0, 0, 255), 1)
    cv2.imshow('Filtered Image', image)
    
def LowerS(val, lower_HSV=lower_HSV):
    global sample_image
    lower_HSV[1] = val
    image, contours, heiarchy = process_image(sample_image, lower_HSV)
    cv2.drawContours(image, contours, -1, (0, 0, 255), 1)
    cv2.imshow('Filtered Image', image)
    
def LowerV(val, lower_HSV=lower_HSV):
    global sample_image
    lower_HSV[2] = val
    image, contours, heiarchy = process_image(sample_image, lower_HSV)
    cv2.drawContours(image, contours, -1, (0, 0, 255), 1)
    cv2.imshow('Filtered Image', image)
    
# Summary of countours found for each species in training set

def contour_summary(labels, filenames, n_contours): 
    df_contours = pd.DataFrame()
    df_contours = pd.DataFrame({'label':labels,'filename':filenames, 'n_contours':n_contours})
    df_contours = df_contours.set_index(['label','filename'])
    df_contours = df_contours.sort_index()
    
    df_summary = pd.DataFrame(index=np.unique(labels), columns=['min num contours','max num contours','frac_with_contours'])
    
    for label in np.unique(labels):
        desc = df_contours.loc[(label)].describe()
        df_summary.loc[(label)]['min num contours'] = desc.loc[('min','n_contours')]
        df_summary.loc[(label)]['max num contours'] = desc.loc[('max','n_contours')]
        df_summary.loc[(label)]['frac_with_contours'] = np.count_nonzero(df_contours.loc[(label)]) / desc.loc[('count','n_contours')]
    
    print(df_summary)


In [5]:
# Read and pre-process data
images_fixed = []
labels_fixed = []
filenames_fixed = []
n_contours_fixed = []

for class_folder_name in os.listdir(TRAIN_DATA_FOLDER):
    class_folder_path = os.path.join(TRAIN_DATA_FOLDER, class_folder_name)
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (200, 200))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image, contours, heiarchy = process_image(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (150,150))
        
        image = image.flatten()
        
        images_fixed.append(image)
        labels_fixed.append(class_folder_name)
        filenames_fixed.append(os.path.basename(image_path))
        n_contours_fixed.append(len(contours))
        
images_fixed = np.array(images_fixed) 
labels_fixed = np.array(labels_fixed)
filenames_fixed = np.array(filenames_fixed)
n_contours_fixed = np.array(n_contours_fixed)

contour_summary(labels_fixed, filenames_fixed, n_contours_fixed)


                          min num contours max num contours frac_with_contours
Black-grass                            1.0             76.0                1.0
Charlock                               1.0             26.0                1.0
Cleavers                               1.0             14.0                1.0
Common Chickweed                       1.0             34.0                1.0
Common wheat                           1.0             38.0                1.0
Fat Hen                                0.0             30.0           0.985263
Loose Silky-bent                       0.0             76.0           0.998471
Maize                                  0.0             27.0           0.895928
Scentless Mayweed                      1.0             40.0                1.0
Shepherds Purse                        1.0             20.0                1.0
Small-flowered Cranesbill              1.0             34.0                1.0
Sugar beet                             1.0          

In [ ]:
# Adjust HSV values on sample image

SAMPLE_IMAGE_PATH = TRAIN_DATA_FOLDER +'/Sugar beet/ffa401155.png'

lower_HSV = np.array([31, 100, 10]) 
upper_HSV = np.array([84, 255, 255])

sample_image = cv2.imread(SAMPLE_IMAGE_PATH, cv2.IMREAD_COLOR)
sample_image = cv2.resize(sample_image, (300, 300))
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2HSV)
processed_image, contours, heiarchy = process_image(sample_image)
# print(count_contours(contours, 1500))

cv2.namedWindow("Filtered Image", cv2.WINDOW_NORMAL)
cv2.drawContours(processed_image, contours, -1, (0, 0, 255), 1)
cv2.imshow("Filtered Image", processed_image)

cv2.createTrackbar('Hue', 'Filtered Image', lower_HSV[0], upper_HSV[0], LowerH)
cv2.createTrackbar('Saturation', 'Filtered Image', lower_HSV[1], upper_HSV[1], LowerS)
cv2.createTrackbar('Value', 'Filtered Image', lower_HSV[2], upper_HSV[2], LowerV)

cv2.waitKey(0)
cv2.destroyAllWindows()

# View samples

fig = plt.figure(figsize=(12,12))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.09, wspace=0.09)
#fig.suptitle('Lower HSV Params, Hue: {} Saturation: {} Value: {}'.format(lower_HSV[0], lower_HSV[1], lower_HSV[2]),  y=1.05, fontsize=14)

for class_folder_name, i in zip(os.listdir(TRAIN_DATA_FOLDER), range(12)):
    class_folder_path = os.path.join(TRAIN_DATA_FOLDER, class_folder_name)
    for image_path, j in zip(glob(os.path.join(class_folder_path, "*.png")), range(5)):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (200, 200))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image, contours, heiarchy = process_image(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = image.flatten()
        ax = fig.add_subplot(12,5, (i*5 + j+1), xticks=[], yticks=[])
        ax.imshow(np.reshape(image, (200,200)), cmap='Greens')
        #print("Species: %s sample %d : %d" % (class_folder_name, j, len(contours)))

fig.tight_layout()
plt.show()